In [389]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import time
import pandas as pd

In [390]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [391]:
!pip install folium

In [392]:
!pip install geopy
from geopy.geocoders import Nominatim


In [393]:

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


In [394]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

html = requests.get(url).content

df_list = pd.read_html(html)

df = df_list[0]

print(df)

df.to_csv('toronto_data.csv')

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                       

In [395]:
df_toronto = pd.read_csv('toronto_data.csv')

In [396]:
df_toronto.head()

,Unnamed: 0,Postal Code,Borough,Neighbourhood
0,0,M1A,Not assigned,Not assigned
1,1,M2A,Not assigned,Not assigned
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [397]:
df_toronto.dropna()

,Unnamed: 0,Postal Code,Borough,Neighbourhood
0,0,M1A,Not assigned,Not assigned
1,1,M2A,Not assigned,Not assigned
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...,...
175,175,M5Z,Not assigned,Not assigned
176,176,M6Z,Not assigned,Not assigned
177,177,M7Z,Not assigned,Not assigned
178,178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [398]:
del df_toronto["Unnamed: 0"]
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


DROP ROWS WITH BOROUGH NOT ASSIGNED

In [399]:
# Get indexes where Borough has value Not assigned
indexNames = df_toronto[df['Borough'] == 'Not assigned'].index
 
# Delete these row indexes from dataFrame
df_toronto.drop(indexNames , inplace=True)

In [400]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### SEARCH FOR ROWS WITH NEIGHBOURHOOD NOT ASSIGNED AND REPLACE WITH BOROUGH

In [401]:
# Replace Neighbourhood that has value Not assigned with Borough
df_toronto.Neighbourhood = df_toronto.Borough.where(df_toronto.Neighbourhood == 'Not assigned', df_toronto.Neighbourhood)

In [402]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [403]:
df_toronto.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [404]:
coordinates = pd.read_csv("https://cocl.us/Geospatial_data")
print(coordinates)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]


In [405]:
coordinates.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
coordinates

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [406]:
latitude_dict = dict(coordinates[['PostalCode', 'Latitude']].values)
df_toronto['Latitude'] = coordinates.PostalCode.map(latitude_dict)

longitude_dict = dict(coordinates[['PostalCode', 'Longitude']].values)
df_toronto['Longitude'] = coordinates.PostalCode.map(longitude_dict)

df_toronto

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.763573,-79.188711
3,M4A,North York,Victoria Village,43.770992,-79.216917
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.773136,-79.239476
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.744734,-79.239476
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.727929,-79.262029
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",NaN,NaN
165,M4Y,Downtown Toronto,Church and Wellesley,NaN,NaN
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",NaN,NaN
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",NaN,NaN


In [407]:
df_toronto.dropna(inplace= True)

#### WE WILL WORK WITH NEIGHBOURHOODS LOCATED IN BOROUGHS THAT CONTAIN THE WORD "TORONTO"

In [408]:
df_neighbourhoods=df_toronto[df_toronto['Borough'].str.contains("Toronto")]
df_neighbourhoods

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.773136,-79.239476
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.727929,-79.262029
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.781638,-79.304302
22,M5C,Downtown Toronto,St. James Town,43.770120,-79.408493
30,M4E,East Toronto,The Beaches,43.737473,-79.464763
31,M5E,Downtown Toronto,Berczy Park,43.739015,-79.506944
40,M5G,Downtown Toronto,Central Bay Street,43.685347,-79.338106
41,M6G,Downtown Toronto,Christie,43.679557,-79.352188
49,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.686412,-79.400049
50,M6H,West Toronto,"Dufferin, Dovercourt Village",43.679563,-79.377529


#### Let's get the geographical coordinates of Toronto.

In [409]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Connection to Foursquare

In [410]:
CLIENT_ID = 'A0AM453CCFJ4AXYHB004LNJDY2GDU2UE2BVUMUZB2BA4KIN5' # your Foursquare ID
CLIENT_SECRET = 'KIPE3R01LXDJXHXXRXLW2IHKRSC4DMKPLWWMXQXOOPMJXGRZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: A0AM453CCFJ4AXYHB004LNJDY2GDU2UE2BVUMUZB2BA4KIN5
CLIENT_SECRET:KIPE3R01LXDJXHXXRXLW2IHKRSC4DMKPLWWMXQXOOPMJXGRZ


#### We chose a neighbourhood to work with

In [411]:
df_toronto.loc[41, 'Neighbourhood']

'Christie'

In [412]:
neighbourhood_latitude = df_toronto.loc[41, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = df_toronto.loc[41, 'Longitude'] # neighborhood longitude value

neighbourhood_name = df_toronto.loc[41, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Christie are 43.6795571, -79.352188.


#### Let's rank the top50 venues in that neighbourhood in a radio of 1km from the center

In [413]:
radius=1000
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            neighbourhood_latitude, 
            neighbourhood_longitude, 
            radius, 
            LIMIT)

results = requests.get(url).json()
results

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:29: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,MenEssentials,Cosmetics Shop,43.677820,-79.351265
1,Cafe Fiorentina,Italian Restaurant,43.677743,-79.350115
2,Pantheon,Greek Restaurant,43.677621,-79.351434
3,Mezes,Greek Restaurant,43.677962,-79.350196
4,La Diperie,Ice Cream Shop,43.677702,-79.352265


In [414]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


#### Let's repeat the same for all neighbourhoods

In [415]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [416]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [417]:
print(toronto_venues.shape)
toronto_venues

(1254, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,Parkwoods,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,Parkwoods,43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,Parkwoods,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,Parkwoods,43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
...,...,...,...,...,...,...,...
1249,Davisville North,43.706748,-79.594054,Economy Rent A Car,43.708471,-79.589943,Rental Car Location
1250,Davisville North,43.706748,-79.594054,Logistics Distribution,43.707554,-79.589252,Bar
1251,Davisville North,43.706748,-79.594054,Saand Rexdale,43.705072,-79.598725,Drugstore
1252,Davisville North,43.706748,-79.594054,PC Garden,43.706539,-79.599359,Garden Center


In [418]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",36,36,36,36,36,36
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",36,36,36,36,36,36
Berczy Park,6,6,6,6,6,6
"Birch Cliff, Cliffside West",2,2,2,2,2,2
...,...,...,...,...,...,...
"Willowdale, Willowdale East",1,1,1,1,1,1
Woburn,7,7,7,7,7,7
Woodbine Heights,1,1,1,1,1,1


In [419]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


In [420]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1249,Davisville North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1250,Davisville North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1251,Davisville North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1252,Davisville North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Let's group by Neighbourhoods and check the average frequency of each venue category

In [421]:

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027778
3,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,"Birch Cliff, Cliffside West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
59,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
60,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
61,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


#### Let's print each Neighbourhood with the top 3 venues

In [422]:
num_top_venues = 3

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0  Sandwich Place  0.08
1    Dessert Shop  0.08
2     Pizza Place  0.08


----Bayview Village----
              venue  freq
0     Jewelry Store  0.25
1  Sushi Restaurant  0.25
2             Trail  0.25


----Bedford Park, Lawrence Manor East----
              venue  freq
0       Coffee Shop  0.22
1  Sushi Restaurant  0.06
2       Yoga Studio  0.03


----Berczy Park----
           venue  freq
0  Grocery Store  0.33
1           Park  0.17
2  Shopping Mall  0.17


----Birch Cliff, Cliffside West----
                  venue  freq
0                  Park   0.5
1                 River   0.5
2  Other Great Outdoors   0.0


----Brockton, Parkdale Village, Exhibition Place----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12


----Caledonia-Fairbanks----
               venue  freq
0     Baseball Field   0.5
1         Food Truck   0.5
2  Accessories Store   0.0



#### Let's put the data in a df

In [423]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Let's create a new df with top 10 venues for each neighbourhood

In [424]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Café,Thai Restaurant,Gym,Sushi Restaurant,Italian Restaurant,Greek Restaurant
1,Bayview Village,Mexican Restaurant,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Smoothie Shop,Beer Bar,Japanese Restaurant,Sandwich Place,Restaurant
3,Berczy Park,Grocery Store,Hotel,Park,Bank,Shopping Mall,Dumpling Restaurant,Drugstore,Donut Shop,Department Store,Dog Run
4,"Birch Cliff, Cliffside West",Park,River,Yoga Studio,Deli / Bodega,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [425]:
neighbourhoods_venues_sorted.dropna()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Café,Thai Restaurant,Gym,Sushi Restaurant,Italian Restaurant,Greek Restaurant
1,Bayview Village,Mexican Restaurant,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Sushi Restaurant,Yoga Studio,Diner,Fast Food Restaurant,Smoothie Shop,Beer Bar,Japanese Restaurant,Sandwich Place,Restaurant
3,Berczy Park,Grocery Store,Hotel,Park,Bank,Shopping Mall,Dumpling Restaurant,Drugstore,Donut Shop,Department Store,Dog Run
4,"Birch Cliff, Cliffside West",Park,River,Yoga Studio,Deli / Bodega,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
...,...,...,...,...,...,...,...,...,...,...,...
58,"Willowdale, Willowdale East",Baseball Field,Yoga Studio,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
59,Woburn,Skating Rink,Beer Store,Athletics & Sports,Curling Ice,Bus Stop,Park,Dumpling Restaurant,Drugstore,Donut Shop,Dessert Shop
60,Woodbine Heights,Park,College Cafeteria,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
61,York Mills West,Bus Line,Park,Sandwich Place,Mobile Phone Shop,Yoga Studio,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


#### Clustering :)

In [426]:
# set number of clusters
kclusters = 5


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 0, 3, 0, 1, 0], dtype=int32)

In [427]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

toronto_merged = df_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M3A,North York,Parkwoods,43.763573,-79.188711,0.0,Electronics Store,Breakfast Spot,Intersection,Bank,Restaurant,Rental Car Location,Medical Center,Mexican Restaurant,Dessert Shop,Donut Shop
3,M4A,North York,Victoria Village,43.770992,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.773136,-79.239476,0.0,Fried Chicken Joint,Gas Station,Bank,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Bakery,Hakka Restaurant,Doner Restaurant,Donut Shop
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.744734,-79.239476,2.0,Playground,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.727929,-79.262029,0.0,Chinese Restaurant,Convenience Store,Coffee Shop,Discount Store,Department Store,Hobby Shop,Dim Sum Restaurant,Diner,Distribution Center,Dog Run


In [429]:
toronto_merged.dropna(inplace= True)

#### Lets visualize!

In [430]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Let's examine the clusters

In [431]:
# CLUSTER 1
toronto_merged.loc[toronto_merged['ClusterLabels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North York,0.0,Electronics Store,Breakfast Spot,Intersection,Bank,Restaurant,Rental Car Location,Medical Center,Mexican Restaurant,Dessert Shop,Donut Shop
3,North York,0.0,Coffee Shop,Korean BBQ Restaurant,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
4,Downtown Toronto,0.0,Fried Chicken Joint,Gas Station,Bank,Thai Restaurant,Athletics & Sports,Caribbean Restaurant,Bakery,Hakka Restaurant,Doner Restaurant,Donut Shop
6,Downtown Toronto,0.0,Chinese Restaurant,Convenience Store,Coffee Shop,Discount Store,Department Store,Hobby Shop,Dim Sum Restaurant,Diner,Distribution Center,Dog Run
9,Scarborough,0.0,Skating Rink,College Stadium,General Entertainment,Café,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
11,North York,0.0,Smoke Shop,Middle Eastern Restaurant,Sandwich Place,Bakery,Shopping Mall,Auto Garage,Concert Hall,Convenience Store,Eastern European Restaurant,Dumpling Restaurant
12,East York,0.0,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Lounge,Department Store,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
13,Downtown Toronto,0.0,Pizza Place,Pharmacy,Italian Restaurant,Noodle House,Gas Station,Intersection,Thai Restaurant,Fast Food Restaurant,Bank,Chinese Restaurant
14,North York,0.0,Park,Playground,Intersection,Yoga Studio,Deli / Bodega,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
17,Etobicoke,0.0,Golf Course,Fast Food Restaurant,Dog Run,Pool,Mediterranean Restaurant,Deli / Bodega,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant


In [433]:
# CLUSTER 2
toronto_merged.loc[toronto_merged['ClusterLabels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,East York,1.0,Park,College Cafeteria,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
23,York,1.0,Park,Convenience Store,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
40,Downtown Toronto,1.0,Park,Convenience Store,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
50,West Toronto,1.0,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
81,Scarborough,1.0,Convenience Store,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
90,Scarborough,1.0,Park,River,Yoga Studio,Deli / Bodega,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
98,York,1.0,Park,Jewelry Store,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


In [434]:
# CLUSTER 3
toronto_merged.loc[toronto_merged['ClusterLabels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,North York,2.0,Playground,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
48,East York,2.0,Lawyer,Playground,Tennis Court,Yoga Studio,Deli / Bodega,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


In [435]:
# CLUSTER 4
toronto_merged.loc[toronto_merged['ClusterLabels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,York,3.0,Baseball Field,Food Truck,Yoga Studio,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
91,North York,3.0,Baseball Field,Yoga Studio,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


In [436]:
# CLUSTER 5
toronto_merged.loc[toronto_merged['ClusterLabels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Etobicoke,4.0,American Restaurant,Motel,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


# END
#### Thanks for reading!